In this Jupyter Notebook we upload and analyze the transactions in Localbitcoins to find what amounts are being used as a crypto vehicle transfer of funds between USD-COP, COP-USD, COP-VED and VED-USD.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime

#Importing the files  with the transactions.  We downloaded the transactions using the API from localbitcoins.  
#The file lb.py contains the code to access this API.

#reading the .csv files and converting them to a Dataframe
df_COP = pd.read_csv("/Users/nicolasmartinez/PycharmProjects/localbitcoins/COP.csv", index_col=False)
df_USD = pd.read_csv("/Users/nicolasmartinez/PycharmProjects/localbitcoins/USD.csv", index_col=False)
df_COP.columns = ['date', 'tid', 'price', 'amount', 'date_formatted','sc_fiat', 'date_yyyy_mm_dd']
df_USD.columns = ['date', 'tid', 'price', 'amount', 'date_formatted','sc_fiat', 'date_yyyy_mm_dd']

#Joining the two dataframes.
result = df_USD.append(df_COP, sort=False)

# Getting the same time frame between currencies.
df_duplicates = result[result["date_yyyy_mm_dd"]>=20141125]

#Creating a dataframe with duplicate values.
df_duplicates = df_duplicates[df_duplicates.duplicated('amount')]

# getting the length of the amount.  We'll work matching transaction up to the 8 decimal.
df_duplicates['lenAm'] = df_duplicates['amount'].astype(str).map(len)
df_duplicates['date_formatted'] = pd.to_datetime(df_duplicates['date_formatted'], format = '%Y-%m-%d')

# working only amounts with 8 decimals - A Satoshi
df_duplicates = df_duplicates[df_duplicates["lenAm"]>=10]
df_duplicates =df_duplicates.sort_values(["date_yyyy_mm_dd", "amount"], ascending=[True, True])# organizing the amounts to compare times
df_duplicates["shift_date"] = df_duplicates["date_formatted"].shift(1)
df_duplicates["shift_amount"] = df_duplicates["amount"].shift(1)

df_duplicates["zero_amt"] = df_duplicates["amount"] - df_duplicates["shift_amount"]
df_duplicates["shift_amount_up"] = df_duplicates["zero_amt"].shift(-1)
df_duplicates["less5"] = df_duplicates["date_formatted"] - df_duplicates["shift_date"]
df_duplicates["less5"] = df_duplicates['less5'] / np.timedelta64(1, 'h')
df_duplicates["less5"] = df_duplicates['less5'].abs()
df_duplicates["Curr_Dest"] = df_duplicates["sc_fiat"].shift(-1)

df_duplicates





In [558]:
#Getting the duplicate transactions on a timeframe that is less than five hours
temp_df = df_duplicates.loc[((df_duplicates['less5'] <=5) &
                         (df_duplicates['zero_amt'] == 0))] 

#Sorting the Dataframe by Date and amount
temp_df =temp_df.sort_values(["date_yyyy_mm_dd", "amount"], ascending=[True, True])
temp_df


,date,tid,price,amount,date_formatted,sc_fiat,date_yyyy_mm_dd,lenAm,shift_date,shift_amount,zero_amt,shift_amount_up,less5,Curr_Dest
778211,1493650267,38378842,1.848380e+03,0.027051,2017-05-01 10:51:07,USD,20170501,10,2017-05-01 10:52:04,0.027051,0.0,0.023369,0.015833,USD
778196,1493880399,38378857,2.019250e+03,0.024762,2017-05-04 02:46:39,USD,20170504,10,2017-05-04 02:46:43,0.024762,0.0,0.000122,0.001111,USD
778200,1493880171,38378853,2.009390e+03,0.024883,2017-05-04 02:42:51,USD,20170504,10,2017-05-04 02:42:55,0.024883,0.0,-0.005030,0.001111,USD
657863,1553905027,41125170,4.017280e+03,0.074677,2019-03-29 20:17:07,USD,20190329,10,2019-03-29 20:19:03,0.074677,0.0,-0.069621,0.032222,USD
492898,1570317966,44315413,8.319330e+03,0.000601,2019-10-05 19:26:06,USD,20191005,10,2019-10-05 19:30:21,0.000601,0.0,0.000000,0.070833,USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
408,1639410298,54722361,1.868996e+08,0.001605,2021-12-13 10:44:58,COP,20211213,10,2021-12-13 10:54:30,0.001605,0.0,0.000005,0.158889,COP
391,1639410980,54722535,1.860514e+08,0.002687,2021-12-13 10:56:20,COP,20211213,10,2021-12-13 11:13:12,0.002687,0.0,0.000000,0.281111,COP
400,1639410390,54722463,1.860514e+08,0.002687,2021-12-13 10:46:30,COP,20211213,10,2021-12-13 10:56:20,0.002687,0.0,0.000001,0.163889,COP
310,1639415111,54723405,1.854001e+08,0.002697,2021-12-13 12:05:11,COP,20211213,10,2021-12-13 12:59:53,0.002697,0.0,0.000010,0.911667,COP


In [559]:
#percentage of transactions of the total that are transfer vehicles
temp_df.size/result.size 

0.08795139480948053

In [482]:
#Converting the Date Column to a date datatype to do comparisons
temp_df['Date'] = temp_df['date'].apply(lambda d: datetime.datetime.fromtimestamp(int(d)).strftime('%Y-%m-%d'))

In [485]:
#Getting the transactions that have origin in one currency and a destination in another currency.
#That gives us the Dataframe with the transactions where crypto was used as a transfer vehicle of funds.
transfer_df = temp_df[temp_df['sc_fiat']!=temp_df['Curr_Dest']] 

In [543]:
#Accesing the historic prices of BTC using the yahoo finance API
import yfinance as yf
import pandas as pd
from datetime import datetime

#set variables
start = datetime(2017, 5, 1)
end = datetime(2021, 12, 13)
symbol = "BTC-USD"

df_y= yf.download(symbol,start=start, end=end,)

dfy_close = df_y.drop(columns=['Open', 'High','Low','Close','Volume'])



[*********************100%***********************]  1 of 1 completed


In [505]:
#Resetting the index to merge the BTC historical price Dataframe with the transaction Dataframe
df_c = dfy_close.reset_index(drop=False)

In [506]:
#Creating a copy of the transfer Dataframe to merge it with the historical price
transfer_df_copy = transfer_df.copy()


In [508]:
#Casting datatype to be able to merge the two dataframes using the Date Column
transfer_df_copy['Date'] = pd.to_datetime(transfer_df_copy['Date'])
df_c['Date'] = pd.to_datetime(df_c['Date'])
df_c = df_c.sort_values('Date')
transfer_df_copy = transfer_df_copy.sort_values('Date')

df_usa_col = pd.merge_asof(transfer_df_copy, df_c, on='Date')

In [509]:
#Calculating the total transfer amount price*quantity
df_usa_col['transfer_USD'] = df_usa_col['amount']*df_usa_col['Adj Close']

In [513]:
#Grouping transactions (Daily)
df_usa_col.groupby('Date').agg({'transfer_USD' : 'sum'})# grouping total transfers from Both Countries by Date (daily)

,transfer_USD
Date,
2020-07-09,2490.880873
2020-07-10,4818.095642
2020-07-11,763.765567
2020-07-12,2527.920833
2020-07-13,2525.232660
...,...
2021-12-03,45.935305
2021-12-04,327.028710
2021-12-05,51.967130


In [549]:
#exporting the describe() statistics to a csv file
df_usa_col_summary.to_csv("summaryTotalTransfersCOPUSD.csv")

In [530]:
#Querying the dataframe to find the transaction that were originated in COP and had USD as a destination
df_COP_to_USD = df_usa_col.query('sc_fiat == "COP"')
df_COP_to_USD

,date,tid,price,amount,date_formatted,sc_fiat,date_yyyy_mm_dd,lenAm,shift_date,shift_amount,zero_amt,shift_amount_up,less5,Curr_Dest,origin_dest,Date,Adj Close,transfer_USD
0,1594343988,44572582,3.285043e+07,0.000304,2020-07-09 20:19:48,COP,20200709,10,2020-07-09 20:42:15,0.000304,0.0,4.240000e-06,0.374167,USD,origin,2020-07-09,9277.967773,2.824306
1,1594334863,44569900,3.260000e+07,0.092025,2020-07-09 17:47:43,COP,20200709,10,2020-07-09 19:14:32,0.092025,0.0,2.990150e-03,1.446944,USD,origin,2020-07-09,9277.967773,853.800716
2,1594347296,44573272,3.259500e+07,0.034668,2020-07-09 21:14:56,COP,20200709,10,2020-07-09 22:33:12,0.034668,0.0,6.895000e-05,1.304444,USD,origin,2020-07-09,9277.967773,321.647566
6,1594339145,44571188,3.285000e+07,0.009132,2020-07-09 18:59:05,COP,20200709,10,2020-07-09 20:45:22,0.009132,0.0,8.810000e-06,1.771389,USD,origin,2020-07-09,9277.967773,84.730298
7,1594335281,44569974,3.226998e+07,0.006198,2020-07-09 17:54:41,COP,20200709,10,2020-07-09 18:40:20,0.006198,0.0,6.400000e-07,0.760833,USD,origin,2020-07-09,9277.967773,57.502154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493,1638306065,54615199,2.265006e+08,0.000132,2021-11-30 16:01:05,COP,20211130,10,2021-11-30 17:05:28,0.000132,0.0,2.170000e-06,1.073056,USD,origin,2021-11-30,57005.425781,7.550369
5500,1638379411,54622778,2.310937e+08,0.000519,2021-12-01 12:23:31,COP,20211201,10,2021-12-01 12:24:23,0.000519,0.0,2.340000e-06,0.014444,USD,origin,2021-12-01,57229.828125,29.717733
5505,1638625496,54649635,2.019998e+08,0.001485,2021-12-04 08:44:56,COP,20211204,10,2021-12-04 11:32:45,0.001485,0.0,5.160000e-06,2.796944,USD,origin,2021-12-04,49200.703125,73.070424
5509,1638892219,54674479,2.014004e+08,0.001490,2021-12-07 10:50:19,COP,20211207,10,2021-12-07 11:06:17,0.001490,0.0,7.400000e-07,0.266111,USD,origin,2021-12-07,50700.085938,75.521327


In [554]:
df_COP_to_USD_sum = df_usa_col.describe()

In [555]:
df_COP_to_USD_sum

,date,tid,price,amount,date_yyyy_mm_dd,lenAm,shift_amount,zero_amt,shift_amount_up,less5,Adj Close,transfer_USD
count,5.514000e+03,5.514000e+03,5.514000e+03,5514.000000,5.514000e+03,5514.0,5514.000000,5514.0,5514.000000,5514.000000,5514.000000,5514.000000
mean,1.608125e+09,4.877862e+07,4.957479e+07,0.008051,2.020457e+07,10.0,0.008051,0.0,-0.000102,0.990109,26154.093540,145.161746
std,1.037960e+07,2.742504e+06,6.255962e+07,0.022810,4.605676e+03,0.0,0.022810,0.0,0.014471,1.174393,17435.804702,384.534086
min,1.594308e+09,4.455542e+07,7.438650e+03,0.000100,2.020071e+07,10.0,0.000100,0.0,-0.857041,0.000278,9132.227539,1.016811
25%,1.599826e+09,4.646884e+07,2.194734e+04,0.000915,2.020091e+07,10.0,0.000915,0.0,0.000001,0.159444,11366.134766,19.730774
50%,1.606375e+09,4.852818e+07,3.679992e+07,0.002248,2.020113e+07,10.0,0.002248,0.0,0.000005,0.493750,18264.992188,45.555207
75%,1.614032e+09,5.089857e+07,6.541481e+07,0.006859,2.021022e+07,10.0,0.006859,0.0,0.000021,1.384514,39266.011719,125.573833
max,1.638984e+09,5.468411e+07,2.450004e+08,0.857143,2.021121e+07,10.0,0.857143,0.0,0.113900,4.997222,67566.828125,15128.933734


In [532]:
# grouping total transfers by Date (daily). Origin COP.  Destination USD.
df_COP_to_USD = df_COP_to_USD.groupby('Date').agg({'transfer_USD' : 'sum'})
df_COP_to_USD

,transfer_USD
Date,
2020-07-09,1905.361223
2020-07-10,1299.230334
2020-07-11,109.446455
2020-07-12,1966.560857
2020-07-13,1934.172763
...,...
2021-11-30,7.550369
2021-12-01,29.717733
2021-12-04,73.070424


In [524]:
#Exporting the Dataframe
df_COP_to_USD.to_csv("transfersfromCOPtoUSD.csv")

In [538]:
#Querying the dataframe to find the transaction that were originated in USD and had COP as a destination
df_USD_to_COP = df_usa_col.query('sc_fiat == "USD"')
df_USD_to_COP

,date,tid,price,amount,date_formatted,sc_fiat,date_yyyy_mm_dd,lenAm,shift_date,shift_amount,zero_amt,shift_amount_up,less5,Curr_Dest,origin_dest,Date,Adj Close,transfer_USD
3,1594314463,44559525,9890.00,0.030334,2020-07-09 13:07:43,USD,20200709,10,2020-07-09 16:37:21,0.030334,0.0,0.000247,3.493889,COP,origin,2020-07-09,9277.967773,281.434813
4,1594307649,44555420,14799.09,0.013514,2020-07-09 11:14:09,USD,20200709,10,2020-07-09 11:15:55,0.013514,0.0,0.000147,0.029444,COP,origin,2020-07-09,9277.967773,125.385611
5,1594331230,44568355,9850.00,0.012183,2020-07-09 17:47:10,USD,20200709,10,2020-07-09 18:43:50,0.012183,0.0,0.000025,0.944444,COP,origin,2020-07-09,9277.967773,113.031069
8,1594314054,44559597,9890.00,0.004044,2020-07-09 13:00:54,USD,20200709,10,2020-07-09 13:36:39,0.004044,0.0,0.000004,0.595833,COP,origin,2020-07-09,9277.967773,37.524648
11,1594321753,44563784,9889.99,0.003033,2020-07-09 15:09:13,USD,20200709,10,2020-07-09 16:04:37,0.003033,0.0,0.000004,0.923333,COP,origin,2020-07-09,9277.967773,28.143509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5507,1638630979,54650568,75863.61,0.002636,2021-12-04 10:16:19,USD,20211204,10,2021-12-04 10:16:52,0.002636,0.0,0.000035,0.009167,COP,origin,2021-12-04,49200.703125,129.708306
5508,1638704091,54656394,47500.07,0.001053,2021-12-05 06:34:51,USD,20211205,10,2021-12-05 07:30:59,0.001053,0.0,0.000018,0.935556,COP,origin,2021-12-05,49368.847656,51.967130
5510,1638890091,54673930,56000.13,0.001786,2021-12-07 10:14:51,USD,20211207,10,2021-12-07 11:04:59,0.001786,0.0,0.000034,0.835556,COP,origin,2021-12-07,50700.085938,90.535650
5512,1638975838,54682720,67100.58,0.000447,2021-12-08 10:03:58,USD,20211208,10,2021-12-08 11:57:01,0.000447,0.0,0.000045,1.884167,COP,origin,2021-12-08,50504.796875,22.580190


In [539]:
#Grouping by day
df_COP_to_USD = df_USD_to_COP.groupby('Date').agg({'transfer_USD' : 'sum'})

In [540]:
df_COP_to_USD

,transfer_USD
Date,
2020-07-09,585.519650
2020-07-10,3518.865309
2020-07-11,654.319113
2020-07-12,561.359976
2020-07-13,591.059897
...,...
2021-12-03,45.935305
2021-12-04,253.958285
2021-12-05,51.967130


In [542]:
#exporting the Dataframe to a csv file
df_COP_to_USD.to_csv("transfersfromUSDtoCOP.csv")